# Wine Quality:
<hr style="border:2px solid black">

## Goal: 
* 
* 
<hr style="border:2px solid black">

# Imports

In [2]:
# Standard ds imports:
import pandas as pd
import numpy as np

# Viz
import matplotlib.pyplot as plt
import seaborn as sns

# Statisitics
from scipy import stats

<hr style="border:2px solid black">

# **Acquire**
Plan --> **Acquire** --> Prepare --> Explore --> Model --> Deliver

In [ ]:
# Acquire

<hr style="border:2px solid black">

# **Prepare**

<hr style="border:2px solid black">

# **Explore**
Plan --> Acquire --> Prepare --> **Explore** --> Model --> Deliver

<hr style="border:2px solid black">

# **Modeling**
Plan --> Acquire --> Prepare --> Explore --> **Model** --> Deliver

<hr style="border:2px solid black">

# **Conclusions**
